# Тематическое моделирование текстов<a

Тематическое моделирование текстов – это метод, который используется для
определения основных тем в больших наборах текстов. Он позволяет выявить
ключевые слова, связанные с каждой темой.

Такой метод используется для анализа комментариев, классификации,
суммаризации, эффективного поиска.

Тематическое моделирование работает так: каждый текст в корпусе имеет
некие темы, которые выражаются через ключевые слова. Метод раскладывает
тексты на темы и определяет связанные слова.

Самый популярный метод – это Латентное распределение Дирихле (LDA):

* Тексты преобразовываются в векторное представление (мешок слов или
TF-IDF)
* Нужно заранее определить количество тем (это самая трудная
часть)
* Обучить модель (мы воспользуемся готовыми реализациями)

In [1]:
# !pip install gensim # установим библиотеки
# !pip install sklearn
# !pip install pyLDAvis

import pandas as pd
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, LdaModel, LsiModel
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

import nltk
nltk.download('stopwords')

KeyboardInterrupt: 

Работать мы будем с отзывами, оставленными пользователями на Яндекс.Картах. Мы работаем с небольшой выборкой, полный репозиторий на
500 тысяч отзывов:
<https://www.kaggle.com/datasets/kyakovlev/yandex-geo-reviews-dataset-2023?resource=download&select=geo-reviews-dataset-2023.csv>

In [ ]:
df = pd.read_csv('geo-reviews-dataset-2023.csv')
print(df.shape)
df = df.sample(frac=0.01, replace=True)
print(df.shape)
df.head()

(500000, 5)
(5000, 5)


,address,name_ru,rating,rubrics,text
233970,"Санкт-Петербург, Кирочная улица, 12",Pita's,5.0,"Быстрое питание;Бар, паб;Кафе","Стильно,вкусно, приятно.\nНеобычная шаверма , ..."
432031,"Волгоград, площадь Дзержинского, 1Б",Читай-город,5.0,Книжный магазин;Магазин канцтоваров;Учебная ли...,Большой магазин много интересной и познаватель...
258575,"Саратовская область, Энгельсский район, село Ш...",Ассамблея,1.0,Гостиница,"Везде грязь, мусор. Никто не убирает, в основн..."
187745,"Москва, Пятницкая улица, 3/4с1",Mitzva Bar,5.0,"Бар, паб;Ресторан","Специфичное место, сложности были найти вход! ..."
126098,"Московская область, Химки, Ленинский проспект, 2А",Родина,5.0,Дом культуры;Концертные и театральные агентства,Ездили на новогоднее представление. Родителей ...


## LDA через scikit

Реализуем первое моделирование через библиотеку scikit. Используем
модель bag-of-words.

In [ ]:
# Подготовка данных, перевод в векторы
documents = df['text'].to_list()
vectorizer = CountVectorizer(stop_words=stopwords.words('russian') + ['очень', 'это', 'всё', 'спасибо', 'место'])
X = vectorizer.fit_transform(documents)

# Применение LDA
num_topics = 10 # это самая сложная часть, сколько же тем есть в отзывах?
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(X)

# Вывод тем и связанных с ними слов
for topic_idx, topic_words in enumerate(lda.components_):
    top_words_idx = topic_words.argsort()[-10:][::-1]
    top_words = [vectorizer.get_feature_names_out()[i] for i in top_words_idx]
    print(f"Тема {topic_idx + 1}: {', '.join(top_words)}")

Тема 1: персонал, вкусно, еда, вкусная, чисто, кафе, обслуживание, рекомендую, отель, понравилось
Тема 2: просто, быстро, хорошая, большое, персонал, время, нужно, понравилось, цены, хочу
Тема 3: вкусно, рекомендую, день, время, понравилось, заведение, персонал, вообще, всем, нам
Тема 4: быстро, всем, нам, рекомендую, ещё, время, цены, просто, день, персонал
Тема 5: просто, персонал, быстро, рекомендую, время, нужно, всем, хорошие, 10, минут
Тема 6: время, просто, чисто, рядом, ещё, нужно, 10, номер, хотя, nв
Тема 7: быстро, всем, мастер, рекомендую, ребята, время, ещё, отличный, нужно, своего
Тема 8: магазин, персонал, выбор, хороший, большой, ассортимент, цены, отличный, рекомендую, вежливый
Тема 9: рекомендую, просто, ещё, день, время, всем, сразу, первый, нравится, салон
Тема 10: быстро, всем, которые, заказ, большое, время, рекомендую, день, цены, советую


Разберем последнюю часть кода:

`lda.components_` представляет матрицу тем, где каждая строка
соответствует теме, а каждый столбец соответствует словам в словаре.

`topic_words.argsort()` возвращает индексы слов в теме в порядке
возрастания значений (весов) этих слов. `[-10:]` выбирает последние 10
индексов (т.е. топ-10 слов с наибольшим весом). `[::-1]` инвертирует
порядок, чтобы получить топ-10 слов с наибольшим весом в порядке
убывания. В результате получаем индексы топ-10 слов с наибольшим весом в
текущей теме.

`vectorizer.get_feature_names_out()[i] for i in top_words_idx` выбирает
слова из словаря, соответствующие индексам топ-10 слов с наибольшим
весом в текущей теме.

## Метод неотрицательного матричного разложения

Теперь преобразуем наши данные через TF-IDF и метод неотрицательного
матричного разложения (NMF, часто используется в анализе данных и
машинном обучении для извлечения скрытых структур из данных.).

![](https://www.researchgate.net/publication/312157184/figure/fig1/AS:448453387001860@1483931027472/Conceptual-illustration-of-non-negative-matrix-factorization-NMF-decomposition-of-a.png)

*Источник:
<https://www.researchgate.net/figure/Conceptual-illustration-of-non-negative-matrix-factorization-NMF-decomposition-of-a_fig1_312157184>*

In [ ]:
# Подготовка данных
documents = df['text'].to_list()
vectorizer = TfidfVectorizer(stop_words=stopwords.words('russian') + ['очень', 'это', 'всё', 'спасибо', 'место'])
X = vectorizer.fit_transform(documents)

# Применение NMF матрицы
num_topics = 10
nmf = NMF(n_components=num_topics, random_state=42) # NMF (Non-Negative Matrix Factorization)
nmf.fit(X)

# Вывод слов для каждой темы
feature_names = vectorizer.get_feature_names_out()
for topic_idx, topic_words in enumerate(nmf.components_):
    top_words_idx = topic_words.argsort()[-10:][::-1]
    top_words = [feature_names[i] for i in top_words_idx]
    print(f"Тема {topic_idx + 1}: {', '.join(top_words)}")

Тема 1: быстро, рекомендую, всем, мастер, качественно, мастера, салон, большое, своего, дела
Тема 2: магазин, хороший, продавцы, вежливые, ассортимент, чисто, свежее, помогут, магазине, приятные
Тема 3: персонал, вежливый, приветливый, отзывчивый, внимательный, чисто, чистота, доброжелательный, расположение, удобное
Тема 4: вкусно, готовят, быстро, заведение, меню, официанты, кафе, ресторан, поесть, порции
Тема 5: вкусная, еда, музыка, атмосфера, приятная, заведение, кухня, кафе, хорошая, свежая
Тема 6: чисто, отель, рядом, номера, номер, понравилось, уютно, номере, кафе, удобно
Тема 7: большой, выбор, товаров, ассортимент, товара, расположение, акции, удобное, парковка, огромный
Тема 8: обслуживание, хорошее, отличное, быстрое, уровне, вежливое, качественное, вкусные, приятная, атмосфера
Тема 9: отличный, магазин, уютный, отель, рекомендую, персонал, отзывчивый, ассортиментом, приятный, помогли
Тема 10: цены, приемлемые, ассортимент, адекватные, качество, хорошие, доступные, отличные,

Кажется, что TF-IDF и NMF справились с классификацией лучше.

## LDA через библиотеку gensim

In [ ]:
from string import punctuation
stop_words_ru = stopwords.words('russian') + ['очень', 'это', 'всё', 'спасибо', 'место']

In [ ]:
# Подготовка данных
documents = df['text'].to_list()
texts = [[word for word in doc.lower().split() if word not in stop_words_ru and word not in punctuation] for doc in documents]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# Обучение модели LDA
num_topics = 5
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

# Вывод тем и связанных с ними слов
for idx, topic in lda_model.print_topics(-1):
    print(f"Тема {idx + 1}: {topic}")

Тема 1: 0.003*"мастер" + 0.002*"своего" + 0.002*"вежливый" + 0.002*"всем" + 0.002*"быстро" + 0.002*"просто" + 0.002*"отличный" + 0.002*"мастера" + 0.002*"время" + 0.002*"персонал."
Тема 2: 0.002*"просто" + 0.002*"рекомендую" + 0.002*"цены" + 0.002*"всем" + 0.002*"персонал" + 0.002*"отличное" + 0.002*"хорошее" + 0.002*"отличный" + 0.001*"большой" + 0.001*"хочется"
Тема 3: 0.003*"просто" + 0.002*"персонал" + 0.002*"ещё" + 0.002*"нам" + 0.002*"время" + 0.002*"еда" + 0.002*"отличный" + 0.002*"вообще" + 0.001*"вкусно" + 0.001*"кафе"
Тема 4: 0.004*"хороший" + 0.003*"ещё" + 0.002*"большой" + 0.002*"цены" + 0.002*"выбор" + 0.002*"магазин" + 0.002*"просто" + 0.002*"отличный" + 0.002*"нужно" + 0.002*"время"
Тема 5: 0.004*"персонал" + 0.002*"просто" + 0.002*"выбор" + 0.002*"большой" + 0.002*"номер" + 0.002*"номере" + 0.002*"рядом" + 0.002*"чисто," + 0.002*"отель" + 0.002*"хороший"


Посмотрим на параметры класса LdaModel:

`corpus`: Это набор текстовых документов, представленных в виде корпуса,
который был предварительно преобразован в числовое представление,
например, с использованием мешка слов или TF-IDF.

`num_topics=num_topics`: Этот аргумент указывает количество тем, которые
вы хотите извлечь из вашего корпуса. num_topics является предварительно
заданным числом тем.

`id2word=dictionary`: Этот аргумент представляет словарь, который
связывает каждый уникальный токен в вашем корпусе с его уникальным
идентификатором. Это позволяет модели LDA понимать, какие слова
присутствуют в вашем корпусе.

`passes=15`: Этот аргумент указывает количество проходов (итераций) по
корпусу при обучении модели LDA. Каждый проход позволяет модели
обновлять параметры, чтобы лучше соответствовать данным. Увеличение
количества проходов может улучшить качество модели, но также может
занять больше времени на обучение.

Также обратим внимание на формат вывода.

`Тема 1: 0.002*"хороший" + 0.002*"приятно" + 0.002*"ещё" + 0.002*"персонал" + 0.002*"магазин" + 0.002*"место," + 0.002*"просто" + 0.001*"всем" + 0.001*"хорошее" + 0.001*"целом"`

Это означает, что тема 1 состоит из слов "хороший" (с весом 0.002),
"приятно" (с весом 0.002), "ещё" (с весом 0.002) и так далее.


## Как же понять, сколько должно быть тем?

Можно воспользоваться измерением perplexity (чем ниже, тем лучше) и
coherence (чем выше, тем лучше).

Подробнее https://habr.com/ru/companies/wunderfund/articles/580230/

In [ ]:
# Подготовка данных
documents = df['text'].to_list()
texts = [[word for word in doc.lower().split() if word not in stop_words_ru and word not in punctuation] for doc in documents]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# Подбор числа тем
for num_topics in range(5, 11):
    lda_model = LdaModel(corpus, id2word=dictionary, num_topics=num_topics, passes=15)
    coherence_model = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
    print(f"Число тем: {num_topics}, Perplexity: {lda_model.log_perplexity(corpus)}, Coherence: {coherence_model.get_coherence():.4f}")

Число тем: 5, Perplexity: -10.439679086542409, Coherence: 0.2277
Число тем: 6, Perplexity: -10.495161837792557, Coherence: 0.2655
Число тем: 7, Perplexity: -10.542811040160725, Coherence: 0.2560
Число тем: 8, Perplexity: -10.570990340843041, Coherence: 0.2925
Число тем: 9, Perplexity: -10.845168778450573, Coherence: 0.3341
Число тем: 10, Perplexity: -11.466327725638699, Coherence: 0.4586


Восемь тем показывают наилучшие результаты.

In [ ]:
# Подготовка данных
documents = df['text'].to_list()
texts = [[word for word in doc.lower().split() if word not in stop_words_ru and word not in punctuation] for doc in documents]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# Обучение модели LDA
num_topics = 8
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

# Вывод тем и связанных с ними слов
for idx, topic in lda_model.print_topics(-1):
    print(f"Тема {idx + 1}: {topic}")

Тема 1: 0.003*"просто" + 0.002*"вообще" + 0.002*"еда" + 0.001*"время" + 0.001*"ещё" + 0.001*"хотя" + 0.001*"особенно" + 0.001*"место," + 0.001*"2" + 0.001*"сказать"
Тема 2: 0.003*"рекомендую" + 0.001*"время" + 0.001*"грамотно" + 0.001*"лучший" + 0.001*"нужно" + 0.001*"красивое" + 0.001*"какое-то" + 0.001*"нет," + 0.001*"просто" + 0.001*"профессионалы"
Тема 3: 0.005*"персонал" + 0.005*"хороший" + 0.004*"цены" + 0.004*"выбор" + 0.003*"рекомендую" + 0.003*"отличный" + 0.003*"просто" + 0.003*"вежливый" + 0.003*"всем" + 0.003*"большой"
Тема 4: 0.003*"ещё" + 0.002*"просто" + 0.002*"кофе" + 0.001*"2" + 0.001*"качество" + 0.001*"лица" + 0.001*"который" + 0.001*"время" + 0.001*"10" + 0.001*"..."
Тема 5: 0.003*"номере" + 0.002*"номер" + 0.002*"рядом" + 0.002*"завтрак" + 0.002*"номера" + 0.002*"отель" + 0.002*"хорошая" + 0.002*"территории" + 0.002*"персонал" + 0.002*"5"
Тема 6: 0.004*"своего" + 0.004*"благодарность" + 0.003*"хочу" + 0.003*"быстро" + 0.003*"всем" + 0.003*"мастер" + 0.002*"просто" 

## Визуализация тематического моделирования

In [ ]:
#!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.162120  0.046808       1        1  21.134099
7     -0.113556 -0.057534       2        1  18.543788
6     -0.083708  0.007062       3        1  15.752855
5      0.053912  0.199444       4        1  13.687583
0      0.065625 -0.023962       5        1   9.327345
4      0.014645 -0.121029       6        1   8.519172
3      0.106457 -0.021956       7        1   7.279174
1      0.118745 -0.028833       8        1   5.755982, topic_info=          Term        Freq       Total Category  logprob  loglift
285      выбор  255.000000  255.000000  Default  30.0000  30.0000
66     магазин  191.000000  191.000000  Default  29.0000  29.0000
991    хороший  281.000000  281.000000  Default  28.0000  28.0000
3122    своего  108.000000  108.000000  Default  27.0000  27.0000
1119    номере   91.000000   91.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
812       хочу    6.549534  117.727910   Topic8  -7.1720  -0.0341
1196   мастера    6.363611   75.900452   Topic8  -7.2008   0.3761
2001  приятные    6.127424   67.332108   Topic8  -7.2386   0.4581
21       также    6.229667  117.020944   Topic8  -7.2220  -0.0781
2962     чисто    5.874510   93.713265   Topic8  -7.2807   0.0853

[616 rows x 6 columns], token_table=       Topic      Freq Term
term                       
9826       8  0.872325   (я
16382      7  0.951306  ...
29992      7  0.948225  000
1670       1  0.017218    1
1670       2  0.361585    1
...      ...       ...  ...
3463       4  0.889375    –
3463       5  0.088937    –
9655       1  0.092102    —
9655       2  0.230254    —
9655       5  0.644712    —

[1386 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 8, 7, 6, 1, 5, 4, 2])

In [ ]:
pyLDAvis.save_html(vis, 'lda.html')